In [2]:
import pickle
import statsmodels.api as sm
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
sns.set_theme()
sns.set_style("darkgrid")
# sns.set_context("paper")
# sns.set_context("talk")

In [3]:
with open('../../results/results_mlp_ratio.P', 'rb') as f:
    results = pickle.load(f)
cortical_results_mlp = results['Cortical']['analysis']

with open('../../results/results_rnn_ratio.P', 'rb') as f:
    results = pickle.load(f)
cortical_results_rnn = results['Cortical']['analysis']


In [4]:
ratio_results_mlp =  cortical_results_mlp['ratio_results']
ratio_results_rnn =  cortical_results_rnn['ratio_results']

In [ ]:
# MLP
ratio_hidds_mlp = ratio_results_mlp['ratio_hidds']
# RNN
ratio_hidds_rnn = ratio_results_rnn['ratio_hidds']

runs = ratio_hidds_mlp.shape[0]
checkpoints = ratio_hidds_mlp.shape[1]
print('runs, checkpoints: ', runs, checkpoints)

fig, axs = plt.subplots(2, 1, figsize=(10,8))
val_name = 'ratio(cong/incong)'
threshold = 1
# MLP
val = ratio_hidds_mlp
df = pd.DataFrame(val, columns= np.arange(checkpoints))
df.insert(0, 'runs', np.arange(runs))
df2 = pd.melt(df, id_vars=['runs'],var_name='steps', value_name=val_name)
# plot
ax = axs[0]
ax = sns.boxplot(x='steps', y=val_name, data=df2, ax=ax)
ax = sns.stripplot(x='steps', y=val_name, data=df2, ax=ax)
ax.set_title('MLP')

# RNN
val = ratio_hidds_rnn
df = pd.DataFrame(val, columns= np.arange(checkpoints))
df.insert(0, 'runs', np.arange(runs))
df2 = pd.melt(df, id_vars=['runs'],var_name='steps', value_name=val_name)
# plot
ax = axs[1]
ax = sns.boxplot(x='steps', y=val_name, data=df2, ax=ax)
ax = sns.stripplot(x='steps', y=val_name, data=df2, ax=ax)
ax.set_title('RNN')

for ax in axs:
    ax.axhline(y=threshold, color='r', linewidth=2)
    ax.axhline(y=-1*threshold, color='r', linewidth=2)

fig.suptitle('Ratio Results', fontweight='bold')
plt.tight_layout()
fig_str = 'ratio_results_both_models_hidds'
fig.savefig(('../../figures/' + fig_str + '.pdf'), 
        bbox_inches = 'tight', pad_inches = 0)


